## __PROJET 5 - IMDB Movie Prediction__ - _*Ludovic & Yasemin*_

####  __SECONDE PARTIE__ : MACHINE LEARNING

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

#On ajoute GridSearchCV/StratifiedKFold pour l'optimisation des hyperparamètres et la validation croisée
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

#Pour le rééquilibrage des classes par suréchantillonnage
from sklearn.utils import resample

In [2]:
#Chargement du dataset nettoyé
pd.set_option("display.max_columns", None)

df = pd.read_csv("./Datas/5000_movies_bis_clean.csv")
df.head(3)

,duration,director_fb_likes,actor_1_fb_likes,gross,num_voted_users,facenumber_in_poster,budget,title_year,aspect_ratio,movie_fb_likes,country_UK,country_USA,other_actors_fb_likes,critic_reviews_ratio,imdb_classification
0,178.0,0.0,1000.0,760505847.0,886204,0.0,237000000.0,2009.0,1.78,33000,False,True,1791.0,0.236739,2
1,169.0,563.0,40000.0,309404152.0,471220,0.0,300000000.0,2007.0,2.35,0,False,True,6000.0,0.243942,2
2,148.0,0.0,11000.0,200074175.0,275868,1.0,245000000.0,2015.0,2.35,85000,True,False,554.0,0.605634,2


In [3]:
#Rappel des infos actualisées de notre dataset nettoyé
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4703 entries, 0 to 4702
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   duration               4703 non-null   float64
 1   director_fb_likes      4703 non-null   float64
 2   actor_1_fb_likes       4703 non-null   float64
 3   gross                  4703 non-null   float64
 4   num_voted_users        4703 non-null   int64  
 5   facenumber_in_poster   4703 non-null   float64
 6   budget                 4703 non-null   float64
 7   title_year             4703 non-null   float64
 8   aspect_ratio           4703 non-null   float64
 9   movie_fb_likes         4703 non-null   int64  
 10  country_UK             4703 non-null   bool   
 11  country_USA            4703 non-null   bool   
 12  other_actors_fb_likes  4703 non-null   float64
 13  critic_reviews_ratio   4703 non-null   float64
 14  imdb_classification    4703 non-null   int64  
dtypes: b

### Modèle de ML : __Random Forest__

In [4]:
#On prépare les données pour l'entrainement
#On supprime la variable cible
X = df.drop(["imdb_classification"], axis=1)
y = df["imdb_classification"]

#On divise le dataset en ensemble entrainement/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#On va maintenant pouvoir entraîner le modèle de classification RF 
#et faire des prédictions sur l'ensemble du test
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

#On évalue ce que retourne le modèle
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

#Rapport détaillé de classification
print(classification_report(y_test, y_pred))


Accuracy: 0.7290116896918172
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        31
           1       0.64      0.46      0.53       272
           2       0.75      0.91      0.82       597
           3       0.89      0.39      0.54        41

    accuracy                           0.73       941
   macro avg       0.57      0.44      0.47       941
weighted avg       0.70      0.73      0.70       941



#### *Commentaires* :
- Ici, le modèle de classification RF prédit correctement la catégorie dans environ 73% du temps sur l'ensemble du dataset
- Cependant, les classes ne sont pas distribuées de manière égale. En effet, les classes "0" et "3" sont mal représentées selon le rapport de classification. La classe 2 est quant à elle bien performée par le modèle, peut être même un peu trop : Faux positifs??

On va tenter d'améliorer le modèle afin d'obtenir des résultats davantage satisfaisants

### Random Forest : __TEST AMELIORATION__

In [5]:
# On va effectuer un rééquilibrage des classes
# avecà nbr fixe limité à 500échantillons
X_y = pd.concat([X, y], axis=1)
upsampled_data = pd.DataFrame()
for class_value in X_y["imdb_classification"].unique():
    class_subset = X_y[X_y["imdb_classification"] == class_value]
    upsampled_class = resample(class_subset, replace=True, n_samples=500, random_state=42)
    upsampled_data = pd.concat([upsampled_data, upsampled_class])

#Une fois le rééquilibrage effectué, on divise de nouveau les données en ensemble entrainement/test
X_upsampled = upsampled_data.drop("imdb_classification", axis=1)
y_upsampled = upsampled_data["imdb_classification"]

X_train, X_test, y_train, y_test = train_test_split(X_upsampled, y_upsampled, test_size=0.2, random_state=42)

In [6]:
#Configuration des différentes combinaisons d'hyperparamètres à tester pour le modèle
param_grid = {
    "n_estimators": [50, 100],
    "max_depth": [10, 20],
    "min_samples_split": [10], 
    "min_samples_leaf": [4]
}
cv = StratifiedKFold(n_splits=3)

#Recherche sur les combinaisons spécifiées plus haut(param_grid)
#Utilisation de la validation croisée pour évaluer chacune d'entre elles
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=cv, scoring="accuracy", verbose=1)

# On réentraine le modèle
grid_search.fit(X_train, y_train)

# Meilleurs paramètres et évaluation
print("Meilleurs paramètres:", grid_search.best_params_)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))


Fitting 3 folds for each of 4 candidates, totalling 12 fits
Meilleurs paramètres: {'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
              precision    recall  f1-score   support

           0       0.84      0.88      0.86       113
           1       0.62      0.66      0.64        88
           2       0.75      0.67      0.71       104
           3       0.95      0.95      0.95        95

    accuracy                           0.80       400
   macro avg       0.79      0.79      0.79       400
weighted avg       0.80      0.80      0.79       400



#### *Commentaires* :
- Avec ajout des paramètres supplémentaires, le modèle a amélioré sa performance et on obtient maintenant 80%
- Les classes semblent être plus équilibrées, même s'il persiste encore des disparités
- Améliorations futures : Ajouter des hyperparam supplémentaires

In [7]:
#Conversion de mon modèle sous pkl pour l'app Flask
import joblib
joblib.dump(best_model, "./models/prediction_model_RF.pkl")

['./models/prediction_model_RF.pkl']

### Modèle ML : Test avec __XGBoost__

In [8]:
from xgboost import XGBClassifier
from sklearn import metrics

xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgbprd = xgb.predict(X_test)
cnf_matrix = metrics.confusion_matrix(y_test, xgbprd)
print(cnf_matrix)
print("Accuracy:", metrics.accuracy_score(y_test, xgbprd))

[[107   5   1   0]
 [  5  64  19   0]
 [  5  20  73   6]
 [  0   1   0  94]]
Accuracy: 0.845


In [9]:
#Conversion de mon modèle sous pkl pour l'app Flask
import joblib
joblib.dump(xgb, "./models/prediction_model_XGB.pkl")

['./models/prediction_model_XGB.pkl']

### XGBoost : __Amélioration__

In [10]:
#Configuration des différentes combinaisons d'hyperparamètres à tester pour le modèle
param_grid = {
    'min_child_weight': [1, 5],
    'gamma': [0.5, 1, 2],
    'subsample': [0.6, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'max_depth': [3, 5]
}
cv = StratifiedKFold(n_splits=3)

#Recherche sur les combinaisons spécifiées plus haut(param_grid)
#Utilisation de la validation croisée pour évaluer chacune d'entre elles
grid_search = GridSearchCV(XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic'),
                           param_grid, cv=cv, scoring="accuracy", verbose=1)

# On réentraine le modèle
grid_search.fit(X_train, y_train)

# Meilleurs paramètres et évaluation
print("Meilleurs paramètres:", grid_search.best_params_)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))

Fitting 3 folds for each of 72 candidates, totalling 216 fits
Meilleurs paramètres: {'colsample_bytree': 1.0, 'gamma': 0.5, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.6}
              precision    recall  f1-score   support

           0       0.89      0.96      0.92       113
           1       0.74      0.69      0.72        88
           2       0.82      0.77      0.79       104
           3       0.95      0.99      0.97        95

    accuracy                           0.86       400
   macro avg       0.85      0.85      0.85       400
weighted avg       0.85      0.86      0.85       400



In [11]:
#Conversion de mon modèle sous pkl pour l'app Flask
import joblib
joblib.dump(best_model, "./models/prediction_model_XGBbest.pkl")

['./models/prediction_model_XGBbest.pkl']